## Calculate Urban Shape Metrics

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os, inspect, logging, importlib, time

In [3]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import math, random

In [4]:
import shapely

In [5]:
# Get reference to GOSTNets
sys.path.append(r'C:\repos\INFRA_SAP')
from infrasap.urban_metrics import *

In [6]:
shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\sample_shps_3.shp"
#shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UBZ_only_FUAs2.shp"
output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool"

In [7]:
# import extent
input_shapes_gpd = gpd.read_file(shpName)

In [8]:
#input_shapes_gpd

In [9]:
# proj
#vars(input_shapes_gpd)
input_shapes_gpd._crs

<Projected CRS: EPSG:32642>
Name: WGS 84 / UTM zone 42N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - N hemisphere - 66°E to 72°E - by country
- bounds: (66.0, 0.0, 72.0, 84.0)
Coordinate Operation:
- name: UTM zone 42N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
# make the GeoDataFrame unprojected
input_shapes_gpd = input_shapes_gpd.to_crs('epsg:4326')
input_shapes_gpd._crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [11]:
input_shapes_gpd

,id,comment,geometry
0,1,test for low roundness,"POLYGON ((69.54914 40.86855, 69.55354 40.87362..."
1,2,"test for high roundness,high proximity, high d...","POLYGON ((69.16911 41.35884, 69.20945 41.38473..."
2,2,"test for low cohesion, low range","POLYGON ((69.49536 41.35673, 69.53582 41.38250..."
3,4,test for low detour,"POLYGON ((69.17387 41.11068, 69.20280 41.11413..."
4,5,"low perimeter index, low girth index","POLYGON ((70.11118 41.04247, 70.15702 41.06534..."
5,6,"test for low proximity, low dispersion index","POLYGON ((69.71784 41.29755, 69.72517 41.30403..."
6,7,"test for low spin index, low depth","POLYGON ((70.17560 41.33755, 70.16453 41.32714..."


In [12]:
start_time = time.time()

In [13]:
#-------------------------------------
# SET UP TEMP WORKSPACE...
TempWS = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\Shape_Metrics_Temp"

# feature grid text file...
gridTxtFile = "%s\\grid.txt" % TempWS

### Loop through each urban extent and calculate its urban metrics. For each urban extent we will reproject it using the UTM zone it belongs in. Then at the end we will unproject it again.

In [14]:
%%time
for index, row in input_shapes_gpd.iterrows():
    print(index)
    metrics = {}

    # creates a temporary GDF for just the row's shape
    temp_gdf = input_shapes_gpd.iloc[[index]]
    
    # finds its correct UTM zone projection and reprojects it
    temp_gdf_proj = project_gdf(temp_gdf)

    A = temp_gdf_proj.iloc[0].geometry.area
    P = temp_gdf_proj.iloc[0].geometry.length
    
    # Equal area circle radius...
    r = (temp_gdf_proj.iloc[0].geometry.area / math.pi)**.5   # radius of equal area circle (circle with area equal to shape area) (derived from A = pi * r squared)
    print(f"print r: {r}")
    p = 2 * math.pi * r         # Equal area circle perimeter

    # LIST OF COORDINATES OF FEATURE VERTICES (for single part features)...
    pntLst = []   # stores feature array...
    subVLst = []

    # Step through exterior part of the feature
    for coord in temp_gdf_proj.iloc[0].geometry.exterior.coords:
        # Print the part number
        #print("coord {}:".format(coord))
        # Step through each vertex in the feature
        # Print x,y coordinates of current point
        #print("{}, {}".format(coord[0], coord[1]))
        X, Y = coord[0], coord[1]     # get point XY  
        subVLst.append([X,Y])  # add XY to list   

    pntLst.append(subVLst)

    # if it has interior polygons
    if len(list(temp_gdf_proj.iloc[0].geometry.interiors)) > 0:
        for poly in list(temp_gdf_proj.iloc[0].geometry.interiors):
            print("new interior polygon")
            subVLst = []
            # Step through each part of the feature
            for coord in poly.coords:
                #print("coord {}:".format(coord))
                # Step through each vertex in the feature
                # Print x,y coordinates of current point
                #print("{}, {}".format(coord[0], coord[1]))
                X, Y = coord[0], coord[1]     # get point XY  
                subVLst.append([X,Y])  # add XY to list 
            #print(subVLst)
            subVLst.reverse()
            #print(subVLst)
            pntLst.append(subVLst)

    # desired shape area in pixels...
    numPix = 20000

    # calculate pixel size...
    cellsize = (A / numPix)**.5

    # get min and max XY values
    minX, minY, maxX, maxY = temp_gdf_proj.iloc[0].geometry.bounds[0],temp_gdf_proj.iloc[0].geometry.bounds[1],temp_gdf_proj.iloc[0].geometry.bounds[2],temp_gdf_proj.iloc[0].geometry.bounds[3]

    # offset grid by half a pixel...
    minX -= cellsize / 2
    maxY += cellsize / 2

    # centroid coordinates
    centroidXY = temp_gdf_proj.iloc[0].geometry.centroid.x,temp_gdf_proj.iloc[0].geometry.centroid.y
    x_offset, y_offset = 0,0
    Xc, Yc = centroidXY[0]-x_offset, centroidXY[1]-y_offset

    # generates a list of points within the shape
    featPntLst = generate_featPntLst(pntLst, minX, minY, maxX, maxY, cellsize, gridTxtFile)

    # NOTE: THE CENTROID IS CURRENTLY USED AS THE CENTER
    # calculate distance of feature points to center...
    D_to_Center, EAC_pix = proximity(featPntLst,Xc,Yc,r)

    # Proximity index (circle / shape)
    # avg distance to center for equal area circle...
    circD = r * (2.0/3.0)
    #print(f"print circD: {circD}")
    #print(f"print D_to_Center: {D_to_Center}")
    ProximityIndex = circD / D_to_Center
    metrics['ProximityIndex'] = ProximityIndex

    # Roundness (exchange-index)
    inArea = EAC_pix * cellsize**2
    areaExchange = inArea / A
    metrics['RoundnessIndex'] = areaExchange

    # Cohesion index 
    # custom tool calculates approx. average interpoint distances between
    # samples of points in shape...
    shp_interD = interpointDistance(featPntLst)

    # average interpoint distance for equal area circle...
    circ_interD = r * .9054

    # cohesion index is ratio of avg interpoint distance of circle to
    # avg interpoint distance of shape...
    CohesionIndex = circ_interD / shp_interD

    metrics['CohesionIndex'] = CohesionIndex

    # Spin index
    # custom tool calculates moment of inertia for shape...
    shpMOI = spin(featPntLst,Xc,Yc)

    # moment of inertia for equal area circle...
    circ_MOI = .5 * r**2

    # calculate spin index (circle / shape)...
    Spin = circ_MOI / shpMOI

    metrics['SpinIndex'] = Spin
    
    # Perimeter index (circle / shape)
    PerimIndex = p / P          # The Perimeter Index
    metrics['PerimIndex'] = PerimIndex

    # Pre-calculations for Depth, Girth, and Dispersion indices

    #print(f"print first 3 of pntLst: {pntLst[0][:3]}")

    # get list of points evenly distributed along perimeter...
    perimPntLst = PerimeterPnts(pntLst, 500)

    #print(f"print first of perimPntLst: {perimPntLst[0]}")

    #------------------------------------------------------------------------------
    # SECTION 7: CALCULATE DISTANCE OF INTERIOR SHAPE POINTS TO PERIMETER POINTS...

    # custom tool calculates distance of each interior point to nearest perimeter point...
    pt_dToE = pt_distToEdge(featPntLst, perimPntLst)

    #print(f"print max pt_dToE: {pt_dToE[-1]}")

    # Depth index
    # custom tool calculates average distance from interior pixels to nearest edge pixels...
    shp_depth = depth(pt_dToE)

    # depth for equal area circle...
    EAC_depth = r / 3

    # calculate depth index (shape / circle)...
    depthIndex = shp_depth / EAC_depth
    metrics['DepthIndex'] = depthIndex

    # Girth index
    # custom tool calculates shape girth (distance from edge to innermost point)
    # and outputs list position of innermost point...
    shp_Girth = girth(pt_dToE)

    # calculate girth index (shape / circle)...
    girthIndex = shp_Girth / r      # girth of a circle is its radius
    #print(f"print shp_Girth: {shp_Girth}")
    #print(f"print r: {r}")
    metrics['GirthIndex'] = girthIndex

    # Dispersion index
    # custom tool calculates average distance between proximate center and edge points...
    dispersionIndex, avgD = dispersion([Xc, Yc], perimPntLst[0])
    metrics['DispersionIndex'] = dispersionIndex

    # Detour index
    # custom tool creates list of points in the exterior polygon shape
    hullPerim = ConvexHull(pntLst[0])

    # calculate detour index (circle / shape)...
    detourIndex = p / hullPerim
    metrics['DetourIndex'] = detourIndex

    # Range index
    # custom tool identifies perimeter points that are farthest apart, outputs
    # distance between furthermost points...
    circumCircD = Range(pntLst[0])

    # calculate range index (circle / shape)
    rangeIndex = (2*r) / circumCircD
    metrics['RangeIndex'] = rangeIndex

    # Put all metrics in a DataFrame
    metrics_scalar = {}
    for k in metrics:
        metrics_scalar[k] = [metrics[k]]
    metrics_df = pd.DataFrame(metrics_scalar)

    # and concatinate it with the row's shape
    new_temp_gdf_proj = pd.concat([temp_gdf_proj.reset_index(drop=True), metrics_df], axis=1)

    #print("print new_temp_gdf_proj")
    #print(new_temp_gdf_proj)

    # make it unprojected
    temp_gdf_proj_4326 = new_temp_gdf_proj.to_crs('epsg:4326')

    # put the results of each row into a new DataFrame
    if index == 0:
        #print("creating output_shapes_gpd_4326")
        output_shapes_gpd_4326 = temp_gdf_proj_4326
    else:
        #print(f"print output_shapes_gpd_4326, and index is {index}")
        #print(output_shapes_gpd_4326)
        #print("to append temp_gdf_proj_4326")
        #print(temp_gdf_proj_4326)
        output_shapes_gpd_4326 = output_shapes_gpd_4326.append(temp_gdf_proj_4326, ignore_index=True)
        #print("output_shapes_gpd_4326 after append")
        #print(output_shapes_gpd_4326)

0
print utm zone: 42
print r: 4761.566335839125
new interior polygon
new interior polygon
print shp_Girth: 2382.8278679114437
print r: 4761.566335839125
1
print utm zone: 42
print r: 9929.291584299026
print shp_Girth: 9231.03994335261
print r: 9929.291584299026
2
print utm zone: 42
print r: 8061.442297358309
new interior polygon
print shp_Girth: 2387.5697414968918
print r: 8061.442297358309
3
print utm zone: 42
print r: 5810.515798929823
print shp_Girth: 1728.3920776336004
print r: 5810.515798929823
4
print utm zone: 42
print r: 10061.70528392576
print shp_Girth: 4175.432615826248
print r: 10061.70528392576
5
print utm zone: 42
print r: 3255.3730869959736
print shp_Girth: 975.3817989244847
print r: 3255.3730869959736
6
print utm zone: 42
print r: 6565.980728899163
print shp_Girth: 2669.8851080692953
print r: 6565.980728899163
Wall time: 2min 42s


In [16]:
output_shapes_gpd_4326

,id,comment,geometry,ProximityIndex,RoundnessIndex,CohesionIndex,SpinIndex,PerimIndex,DepthIndex,GirthIndex,DispersionIndex,DetourIndex,RangeIndex
0,1,test for low roundness,"POLYGON ((69.54914 40.86855, 69.55354 40.87362...",0.794778,0.61645,0.787300,0.597001,0.403935,0.396848,0.500429,0.762113,0.707102,0.623930
1,2,"test for high roundness,high proximity, high d...","POLYGON ((69.16911 41.35884, 69.20945 41.38473...",0.996278,0.96065,0.996426,0.990284,0.982114,0.969761,0.929678,0.960799,0.959956,0.940122
2,2,"test for low cohesion, low range","POLYGON ((69.49536 41.35673, 69.53582 41.38250...",0.666781,0.48395,0.699615,0.488066,0.501422,0.381340,0.296172,0.960098,0.550037,0.763271
3,4,test for low detour,"POLYGON ((69.17387 41.11068, 69.20280 41.11413...",0.432947,0.30425,0.446119,0.163662,0.489409,0.389886,0.295378,0.527382,0.485085,0.325305
4,5,"low perimeter index, low girth index","POLYGON ((70.11118 41.04247, 70.15702 41.06534...",0.798204,0.62215,0.778797,0.595686,0.411984,0.361104,0.414983,0.717967,0.613894,0.622679
5,6,"test for low proximity, high dispersion","POLYGON ((69.71784 41.29755, 69.72517 41.30403...",0.641670,0.48005,0.639682,0.389077,0.337304,0.299787,0.299622,0.665529,0.583983,0.495162
6,7,"test for spin, low depth","POLYGON ((70.17560 41.33755, 70.16453 41.32714...",0.558291,0.30625,0.566161,0.310865,0.398210,0.362155,0.406624,0.636857,0.481539,0.528113


In [17]:
# save as shapefile
output_shapes_gpd_4326.to_file(r"C:\repos\INFRA_SAP\Notebooks\output_sample_shps3.shp")